In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import numpy as np
import requests as re
import time

In [2]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome('./chromedriver', options=chrome_options)

/var/folders/mr/pjjzz9nd583cq7jrp7gg9jvr0000gn/T/ipykernel_55431/74980914.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver', options=chrome_options)


In [3]:
wait_time = 5

def view_page(driver, page_name):
    urls = {'home':"https://usrwtchtakehome.onrender.com/",
            'horses':"https://usrwtchtakehome.onrender.com/horses",
            'races':"https://usrwtchtakehome.onrender.com/races"
            }
    driver.get(urls[page_name])
    time.sleep(wait_time)


def search_input_field(driver, input_id, id):
    # horse_id_field, race_id_field
    search_field = driver.find_element(by='id', value=input_id)
    search_field.clear()
    search_field.send_keys(id)
    search_field.send_keys(Keys.RETURN)
    time.sleep(wait_time)


def click_button(driver, button_id):
    # show_gate, show_place, show_class, show_distance, show_earnings, search_race, show_individual_race
    button = driver.find_element(by='id', value=button_id)
    button.click()
    time.sleep(wait_time)


def get_country_options(driver):
    country_dd = driver.find_element(by='id', value='country')
    return [c.text for c in country_dd.find_elements_by_tag_name('option')]


def select_dropdown(driver, dd_id, selection=None):
    # country, cities, class, distance
    dd = driver.find_element(by='id', value=dd_id)
    if selection is None:
        selection = np.random.choice([c.text for c in dd.find_elements_by_tag_name('option')], 1)[0]
    Select(dd).select_by_visible_text(selection)
    time.sleep(wait_time)
    return selection


def get_race_ids_from_dropdown_search(country, city='', race_class='', distance=''):
    resp = re.post(f'https://usrwtchtakehome.onrender.com/search_race_info?country={country}&city={city}&class={race_class}&distance={distance}')
    return resp.json()['data'][0]['cells']['values'][0]

In [4]:
def random_strategy_1(driver):
    view_page(driver, 'home')
    view_page(driver, 'horses')
    for i in range(np.random.randint(1, np.random.randint(2, 5))):
        search_input_field(driver, 'horse_id_field', int(np.random.choice(range(0, 358847), 1)[0]))
        for i in range(np.random.randint(1, np.random.randint(3, 10))):
            click_button(driver, np.random.choice(['show_gate', 'show_place', 'show_class', 'show_distance', 'show_earnings'], 1)[0])


def random_strategy_2(driver):
    view_page(driver, 'home')
    view_page(driver, 'horses')
    for i in range(np.random.randint(1, 3)):
        search_input_field(driver, 'horse_id_field', int(np.random.choice(range(0, 358847), 1)[0]))
        for i in range(np.random.randint(0, 2)):
            click_button(driver, np.random.choice(['show_gate', 'show_place', 'show_class', 'show_distance', 'show_earnings'], 1)[0])

    view_page(driver, 'races')

    for i in range(np.random.randint(1, 6)):
        select_dropdown(driver, 'country')
        query_fields = np.random.choice(['country', 'cities', 'class', 'distance'], np.random.randint(0, 5), False)
        query_params = {}
        for q in query_fields:
            query_params[q] = select_dropdown(driver, q)
        click_button(driver, 'search_race')

        if np.random.rand() > 0.6:
            search_param_getter = lambda d, k: '' if d.get(k) is None else d.get(k)
            race_ids = get_race_ids_from_dropdown_search(search_param_getter(query_params, 'country')
                                                     , city=search_param_getter(query_params, 'city')
                                                     , race_class=search_param_getter(query_params, 'class')
                                                     , distance=search_param_getter(query_params, 'distance'))
            if len(race_ids) > 0:
                search_input_field(driver, 'race_id_field', np.random.choice(race_ids, 1)[0])
                click_button(driver, 'show_individual_race')

In [ ]:
for i in range(50):
    if np.random.rand() >= 0.5:
        print('strategy 1')
        random_strategy_1(driver)
    else:
        print('strategy 2')
        random_strategy_2(driver)

    time.sleep(10*60)

strategy 1
strategy 2


/usr/local/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead")


strategy 1
strategy 2
strategy 1
strategy 1
